## Vertex AI Search - Langchain API를 통해서 검색

Feedback : shins777@gmail.com

이 Colab 은 대부분의 고객의 요구사항이면서, 많은 유즈케이스에서의 근간이 되는 지식검색에 대한 예제입니다.
Gronding service 로 Vertex AI Search와 Langchain 을 연동해서 정보를 검색하고 해당 검색된 정보를 Gemini 를 통해서 추론해서 답해주는 형태의 예제입니다.

* Vertex AI Search Manual references
    * https://cloud.google.com/enterprise-search?hl=ko
    * https://cloud.google.com/generative-ai-app-builder/docs/try-enterprise-search


이 소스에서 지식 검색의 대상은 "정보통신 진흥 및 융합 활성화 등에 관한 특별법" 이며 해당 특별법은 아래 사이트에서 다운받았습니다.
https://www.law.go.kr/

* 라이브러리 설치 필요 : google-cloud-discoveryengine
* 설치후 커널 재시작해주세요.

In [ ]:
%pip install --upgrade --quiet langchain langchain-core langchain-google-vertexai google-cloud-discoveryengine

In [ ]:
from IPython.display import display, Markdown

### GCP 사용자 인증 / 환경설정

GCP 인증방법은 아래와 URL 정보를 참고하여 GCP에 접근 하는 환경을 구성해야 합니다. 
* https://cloud.google.com/docs/authentication?hl=ko
* 자세한 정보는 [README.md](https://github.com/shins777/google_gen_ai_sample/blob/main/notebook/gemini/README.md) 파일 참고하세요.

In [ ]:
#  아래 코드는 Colab 환경에서만 실행해주세요. 다른 환경에서는 동작하지 않습니다.
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

### GCP 프로젝트 및 리전 설정
본인의 GCP 환경에 맞게 아래 설정을 구성하세요.  
* 구글의 최신버전인 gemini pro 사용을 권고드립니다.   
* 만일, 기본 버전 text bison 을 사용하려한다면, 참조하는 class 가 다르므로 주의하세요.  
* 현재 Gemini는 한국리전(asia-northeast3)을 통해서 접근이 가능합니다.
* 아래 DATA_STORE_ID는 Vertex AI Search를 구성 후 Data store id 의 값입니다. 

In [ ]:
PROJECT_ID="ai-hangsik"
REGION="asia-northeast3"
DATA_STORE_ID = "hackathon-ds_1709838247983"
MODEL = "gemini-pro"

### Vertex AI Search 검색
아래 해당 문서에 대해서 Vector 검색을 위해서는 Vertex AI Search 내에서 해당 검색 처리에 대한 구성을 해야 합니다.  
해당 솔루션의 메뉴얼은 아래와 같습니다.
* https://cloud.google.com/generative-ai-app-builder/docs/try-enterprise-search

해당 Vertex AI 구성이 완료되면 아래 Langchain 함수를 통해서 처리를 구성할 수 있습니다.
*   https://python.langchain.com/docs/integrations/retrievers/google_vertex_ai_search



In [ ]:
from langchain.retrievers import (
    GoogleVertexAIMultiTurnSearchRetriever,
    GoogleVertexAISearchRetriever,
)

DOC_REGION="global"

retriever = GoogleVertexAISearchRetriever(
    project_id=PROJECT_ID,
    location_id=DOC_REGION,
    data_store_id=DATA_STORE_ID,
    max_documents=3,
    # max_extractive_answer_count=2,
    # get_extractive_answers=True,
    max_extractive_segment_count=1,
    get_extractive_segments=True,
    query_expansion_condition=2,
    engine_data_type = 0
)


#### Vertex AI Search 실행

In [ ]:
query = "특별법에서 민간 부문의 양자정보통신기술개발 지원에 대해서 상세하게 설명해주세요."

result = retriever.get_relevant_documents(query)

context ={}

for doc in result:
    print(doc.json)
    context[doc.metadata['source']] = doc.page_content
    print("\n\n======================")
    print(doc.metadata['source'])
    print(doc.page_content)

print(context)


### Gemini Pro 실행 - Vertex AI Search as a Grounding Service

*   VertexAI API : https://api.python.langchain.com/en/stable/llms/langchain_google_vertexai.llms.VertexAI.html#langchain_google_vertexai.llms.VertexAI

In [ ]:
from langchain_google_vertexai.llms import VertexAI

gemini_pro = VertexAI( model_name = MODEL,
                  project=PROJECT_ID,
                  location=REGION,
                  verbose=True,
                  streaming=False,
                  temperature = 0.2,
                  top_p = 1,
                  top_k = 40
                 )

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

prompt = PromptTemplate.from_template("""

  당신은 법률을 상담해주는 AI 어시스턴트입니다.
  아래 Question 에 대해서 반드시 Context에 있는 개별 내용을 기반으로 단계적으로 추론해서 근거를 설명하고 쉽게 답변해주세요.
  Context : {context}
  Question : {question}

  """)

prompt = prompt.format(context=context,
                       question=query)

print(f"Prompt : {prompt}")

response = gemini_pro.invoke(prompt)
display(Markdown(response))

